In [3]:
import re, time, json, logging, os, requests
import numpy as np
import pandas as pd
from pathlib import Path
from difflib import get_close_matches
from nltk.stem import PorterStemmer #stemming
from dataclasses import dataclass, asdict
from datetime import datetime

from typing import Dict, Any, Optional, Set, List, Tuple

In [4]:
from dotenv import load_dotenv

import openai
from openai import OpenAI

from getpass import getpass

In [ ]:
chatContext = [ # This is a list of dictionaries
    { "role": "system",
    "content": """You are an intent router for healthcare queries.
Classify the user's message into EXACTLY ONE of:
- Scheduling
- Q&A
(Use "User_Decision" if it is perfectly balanced and cannot be decided, or if neither fits clearly.)

Return ONLY a strict JSON object. All keys must be lowercase.
No extra text before or after the JSON.

Schema (exact keys and types):
{
  "schema_version": "1.0",            // string
  "intent": "scheduling|q&a|user_decision",  // string, one of these, lowercase
  "confidence": 0.0,                  // number in [0,1]
  "rationale": "short reason",        // string, ≤ 20 words
  "counts": {"scheduling": 0, "qna": 0},  // object with two integers (0 or positive)
  "evidence": [],                     // list of short strings, each ≤ 3 words
  "source": "llm",                    // string: always "llm" here
  "raw_text": "..."                   // echo user message
}

Rules:
- Be deterministic and concise. Temperature is 0.
- If the message clearly asks to book/change/cancel an appointment: intent = "scheduling".
- If the message asks for general info/policy/medication/hours: intent = "q&a".
- If votes tie or unclear: intent = "user_decision".

Examples:
- "I need to book an appointment"  -> intent="scheduling"
- "What are the clinic hours?"     -> intent="q&a"
"""
}]
chatContext

[{'role': 'system',
  'content': 'You are an intent router for healthcare queries.\nClassify the user\'s message into EXACTLY ONE of:\n- Scheduling\n- Q&A\n(Use "User_Decision" if it is perfectly balanced and cannot be decided, or if neither fits clearly.)\n\nReturn ONLY a strict JSON object. All keys must be lowercase.\nNo extra text before or after the JSON.\n\nSchema (exact keys and types):\n{\n  "schema_version": "1.0",            // string\n  "intent": "scheduling|q&a|user_decision",  // string, one of these, lowercase\n  "confidence": 0.0,                  // number in [0,1]\n  "rationale": "short reason",        // string, ≤ 20 words\n  "counts": {"scheduling": 0, "qna": 0},  // object with two integers (0 or positive)\n  "evidence": [],                     // list of short strings, each ≤ 3 words\n  "source": "llm",                    // string: always "llm" here\n  "raw_text": "..."                   // echo user message\n}\n\nRules:\n- Be deterministic and concise. Temperatur

In [ ]:
os.environ.pop("OPENAI_API_KEY", None)

key = getpass("Enter the OpenAI API key: ")
with open("/content/.env", "w") as f:
  f.write("OPENAI_API_KEY=" + key + "\n")

#load env
loadedENV = load_dotenv("/content/.env")

api_key = os.getenv("OPENAI_API_KEY")

Enter the OpenAI API key: ··········


===

===

In [ ]:
intent = {
    "Scheduling": [
        "schedule","appointment","book","doctor","meeting","make","time","cancel", "update",
        "reschedule","date","available","availability","department","visit",
        "slot","slots","openings","see","see a doctor","follow-up","check-in"
    ],
    "Q&A": [
        "question","query","general","answer","ask","hours","how","what","which","when",
        "side effect","side effects","dosage","dose","instruction","policy","coverage",
        "benefit","copay","cost","price","refill","medication","medicine","symptom","faq"
    ]
}

In [5]:
disclaimer_txt = "This is general medical information; if you have an emergency situation, please dial 911."

# define class

class AgentLogic1:

    def __init__(self, intent_map: Dict[str, List[str]], disclaimer: str, scheduling=None, qna=None):

      self.intent = intent_map
      self.disclaimer = disclaimer

      self.stemmer = PorterStemmer()
      self.sch_intent_scanned = [self.stemmer.stem(word1) for word1 in self.intent['Scheduling']]
      self.qna_intent_scanned = [self.stemmer.stem(word2) for word2 in self.intent['Q&A']]

      self.scheduling = scheduling
      self.qna = qna


    def token_word(self, en_info: str):
      tokens_from_info = re.findall(r"[A-Za-z]+(?:'[A-Za-z]+)?|[0-9]+", en_info or "")
      token1b1 = [t.lower() for t in tokens_from_info]
      return token1b1

  #stemming
    def words_subwords_scanner(self, token: str) -> str:
      """
      Use subword tokenization and word match checking to detect the intention of the sentence: scheduling or general Q&A?
      """
      task = None
      subword = self.stemmer.stem(token)

      if subword in self.sch_intent_scanned or token in self.intent['Scheduling']:
        task = "Scheduling"
        return task
      if subword in self.qna_intent_scanned or token in self.intent['Q&A']:
        task = "Q&A"
        return task

      return None


    def sentence_scanner(self, en_info: str):
      """
      scan the entire sentence to look for keyword(s).
      """
      individual_tokens = self.token_word(en_info)
      scanned_results = {
          "schema_version": "1.0", #0
          "intent": None,  #1
          "confidence": None,    #2
          "rationale": None, #3
          "counts": {"scheduling": 0, "qna": 0}, #4
          "evidence": [], #5
          "source": "stemming rule", #6
          "raw_text": en_info, #7
          }

      for i, one_token in enumerate(individual_tokens):
        task_name = self.words_subwords_scanner(one_token)
        if task_name is not None:
          match = True
          one_result = {"index": i, "keyword": one_token, "specific_task": task_name, "match": match}
          scanned_results["evidence"].append(one_result)

      #scheduling_count & qna_count
      scanned_results["counts"]["scheduling"] = sum(1 for r in scanned_results["evidence"] if r.get("specific_task") == "Scheduling")
      scanned_results["counts"]["qna"] = sum(1 for r in scanned_results["evidence"] if r.get("specific_task") == "Q&A")
      scheduling_count = scanned_results["counts"]["scheduling"]
      qna_count = scanned_results["counts"]["qna"]

      #confidence
      total = max(1, scheduling_count+qna_count)
      distanceA = abs(scheduling_count - qna_count)
      confi_score = 0.5 + 0.5 * (distanceA / total)

      scanned_results["confidence"] = confi_score


      if scheduling_count > qna_count:
        scanned_results["intent"] =  "scheduling"
        scanned_results["rationale"] = "keyword votes favor scheduling"
      elif scheduling_count < qna_count:
        scanned_results["intent"] = "qna"
        scanned_results["rationale"] = "keyword votes favor qna"
      elif scheduling_count == qna_count:
        scanned_results["intent"] = "user_decision"
        scanned_results["rationale"] = "keyword votes equal (>=0) or unusual text, favor user's decision"

      return scanned_results

    def llm_intent_judge0(self, context1: List[Dict[str,str]], en_info: str) -> str: # Chat Completions
      """
      Try to have LLM to do the work.
      """
      api_key = os.getenv("OPENAI_API_KEY")

      if not api_key:
        raise RuntimeError("API key needed")

      client= OpenAI(api_key=api_key)

      instruction=list(context1)
      instruction.append({"role": "user", "content": f"message: {en_info}"})

      response= client.chat.completions.create(model="gpt-4o-mini", messages=instruction, temperature=0)
      return response.choices[0].message.content

    def llm_intent_judge(self, context1: List[Dict[str,str]], en_info: str) -> str: # response API - selected
      """
      Try to have LLM to do the work.
      """
      api_key = os.getenv("OPENAI_API_KEY")

      if not api_key:
        raise RuntimeError("API key needed")

      client= OpenAI(api_key=api_key)

      instruction=list(context1)
      instruction.append({"role": "user", "content": f"message: {en_info}"})

      try:
        response= client.responses.create(model="gpt-4o-mini", input=instruction, temperature=0)
        try:
          llm_output = json.loads(response.output_text) #make it python dictionary
        except Exception:
          llm_output = {
              "schema_version": "1.0",
              "intent": "user_decision",
              "confidence": 0.5,
              "rationale": "invalid llm json",
              "counts": None,
              "evidence": None,
              "source": "llm",
              "raw_text": en_info
              }

        return llm_output
      except Exception:
        return {
              "schema_version": "1.0",
              "intent": "user_decision",
              "confidence": 0.5,
              "rationale": "invalid llm json",
              "counts": None,
              "evidence": None,
              "source": "llm",
              "raw_text": en_info
              }
#

    def hybrid_routing_judge(self, context1: List[Dict[str,str]], en_info: str) -> Dict:
      if not en_info:
        return {
            "schema_version": "1.0",
            "intent": "user_decision",
            "confidence": 0.5,
            "rationale": "empty or missing text",
            "counts": {"scheduling": 0, "qna": 0},
            "evidence": [],
            "source": "guard",
            "raw_text": en_info or ""
        }

      stem_result = self.sentence_scanner(en_info)

      if stem_result["confidence"] < 0.6:
        final_routing = self.llm_intent_judge(context1=context1, en_info=en_info)
      else:
        final_routing = stem_result

      return final_routing

    def route_decision(self, context1: List[Dict[str,str]], en_info: str) -> Dict:
      route_judgment = self.hybrid_routing_judge(context1, en_info)

      intent = str(route_judgment.get("intent", "user_decision"))
      try:
        confidence = float(route_judgment.get("confidence", 0.5))
      except Exception:
        confidence = 0.5

      if confidence >= 0.6:
        if intent == "scheduling":
          next_service = "appointment_service"
          action = "book_appointment"

        elif intent == "qna":
          next_service = "qna_service"
          action = "answer_question"
        else:
          next_service = "frontend"
          action = "ask_user_decision"

      else:
          next_service = "frontend"
          action = "ask_user_decision"

      return {
        "intent": route_judgment["intent"],
        "confidence": route_judgment["confidence"],
        "next_service": next_service,
        "action": action,
        "payload": {
          "text": en_info,
          "language": route_judgment.get("language", "English")
    }
    }

    #




### test

In [ ]:
agentlogic = AgentLogic1(intent, disclaimer_txt)

In [ ]:
#simple test


sentence_test1 = "I need help scheduling an appointment with doctor Wu ASAP."
sentence_test2 = "I want to ask what is the side effect of Tylenol?"
sentence_test3 = "Can you help me check the appointment I already made and ask hermatology department when would they be good to see me?"


testsentence1 = agentlogic.sentence_scanner(sentence_test1)
print(f"This is the result of test sentence 1: {testsentence1}")
print("\n")

testsentence2 = agentlogic.sentence_scanner(sentence_test2)
print(f"This is the result of test sentence 2: {testsentence2}")
print("\n")

testsentence3 = agentlogic.sentence_scanner(sentence_test3)
print(f"This is the result of test sentence 3: {testsentence3}")

This is the result of test sentence 1: {'schema_version': '1.0', 'intent': 'scheduling', 'confidence': 1.0, 'rationale': 'keyword votes favor scheduling', 'counts': {'scheduling': 3, 'qna': 0}, 'evidence': [{'index': 3, 'keyword': 'scheduling', 'specific_task': 'Scheduling', 'match': True}, {'index': 5, 'keyword': 'appointment', 'specific_task': 'Scheduling', 'match': True}, {'index': 7, 'keyword': 'doctor', 'specific_task': 'Scheduling', 'match': True}], 'source': 'stemming rule', 'raw_text': 'I need help scheduling an appointment with doctor Wu ASAP.'}


This is the result of test sentence 2: {'schema_version': '1.0', 'intent': 'qna', 'confidence': 1.0, 'rationale': 'keyword votes favor qna', 'counts': {'scheduling': 0, 'qna': 2}, 'evidence': [{'index': 3, 'keyword': 'ask', 'specific_task': 'Q&A', 'match': True}, {'index': 4, 'keyword': 'what', 'specific_task': 'Q&A', 'match': True}], 'source': 'stemming rule', 'raw_text': 'I want to ask what is the side effect of Tylenol?'}


This i

In [ ]:
sentence_test1 = "I need help scheduling an appointment with doctor Wu ASAP."
sentence_test2 = "I want to ask what is the side effect of Tylenol?"
sentence_test3 = "Can you help me check the appointment I already made and ask hermatology department when would they be good to see me?"

print("Scheduling example: ")
print(agentlogic.llm_intent_judge(chatContext, sentence_test1))
print("\n")

print("Q&A example: ")
print(agentlogic.llm_intent_judge(chatContext, sentence_test2))
print("\n")

print("User Decision example: ")
print(agentlogic.llm_intent_judge(chatContext, sentence_test3))

Scheduling example: 
{'schema_version': '1.0', 'intent': 'scheduling', 'confidence': 1.0, 'rationale': 'User explicitly requests to schedule an appointment.', 'counts': {'scheduling': 1, 'qna': 0}, 'evidence': ['schedule appointment'], 'source': 'llm', 'raw_text': 'I need help scheduling an appointment with doctor Wu ASAP.'}


Q&A example: 
{'schema_version': '1.0', 'intent': 'q&a', 'confidence': 0.9, 'rationale': 'User is seeking information about medication.', 'counts': {'scheduling': 0, 'qna': 1}, 'evidence': ['side effect', 'Tylenol'], 'source': 'llm', 'raw_text': 'I want to ask what is the side effect of Tylenol?'}


User Decision example: 
{'schema_version': '1.0', 'intent': 'scheduling', 'confidence': 0.8, 'rationale': 'User is checking an appointment and seeking scheduling info.', 'counts': {'scheduling': 1, 'qna': 0}, 'evidence': ['check appointment', 'ask department'], 'source': 'llm', 'raw_text': 'Can you help me check the appointment I already made and ask hermatology depar

In [ ]:
llmtext1 = "I need to book an appointment next Tuesday."
llmtext2 = "What are the clinic hours?"

print("Scheduling example: ")
print(agentlogic.llm_intent_judge(chatContext, llmtext1))

print("Q&A example: ")
print(agentlogic.llm_intent_judge(chatContext, llmtext2))

Scheduling example: 
{'schema_version': '1.0', 'intent': 'scheduling', 'confidence': 1.0, 'rationale': 'User clearly wants to book an appointment.', 'counts': {'scheduling': 1, 'qna': 0}, 'evidence': ['book appointment'], 'source': 'llm', 'raw_text': 'I need to book an appointment next Tuesday.'}
Q&A example: 
{'schema_version': '1.0', 'intent': 'q&a', 'confidence': 1.0, 'rationale': 'User is asking for information.', 'counts': {'scheduling': 0, 'qna': 1}, 'evidence': ['clinic hours'], 'source': 'llm', 'raw_text': 'What are the clinic hours?'}


In [ ]:
agent = AgentLogic1(intent, disclaimer_txt)

cases = [
  "I need help scheduling an appointment with doctor Wu ASAP.",
  "What are the clinic hours?",
  "Can you check my appointment and also tell me when hematology can see me?",
  ""
]

for s in cases:
    print("INPUT:", s)
    rule_result = agent.sentence_scanner(s)
    print("RULE RESULT:", json.dumps(rule_result, indent=2))
    route_plan = agent.route_decision(context1=[], en_info=s)
    print("ROUTE PLAN:", json.dumps(route_plan, indent=2))
    print("-"*90)


INPUT: I need help scheduling an appointment with doctor Wu ASAP.
RULE RESULT: {
  "schema_version": "1.0",
  "intent": "scheduling",
  "confidence": 1.0,
  "rationale": "keyword votes favor scheduling",
  "counts": {
    "scheduling": 3,
    "qna": 0
  },
  "evidence": [
    {
      "index": 3,
      "keyword": "scheduling",
      "specific_task": "Scheduling",
      "match": true
    },
    {
      "index": 5,
      "keyword": "appointment",
      "specific_task": "Scheduling",
      "match": true
    },
    {
      "index": 7,
      "keyword": "doctor",
      "specific_task": "Scheduling",
      "match": true
    }
  ],
  "source": "stemming rule",
  "raw_text": "I need help scheduling an appointment with doctor Wu ASAP."
}
ROUTE PLAN: {
  "intent": "scheduling",
  "confidence": 1.0,
  "next_service": "appointment_service",
  "action": "book_appointment",
  "payload": {
    "text": "I need help scheduling an appointment with doctor Wu ASAP.",
    "language": "English"
  }
}
--------